# Day 23: Opening the Turing Lock

[*Advent of Code 2015 day 23*](https://adventofcode.com/2015/day/23) and [*solution megathread*](https://www.reddit.com/3xxdxt)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/23/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F23%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML


HTML(downloaded['part1'])

## Comments

It's 2022-11-26, soon time for this year's AoC, and I'm fiddling around with Jupyter and visualization methods to achieve the best experience... Currently in Notebook with vim bindings and a theme - it's not perfect but at least interesting.
For this problem, I'll attempt an immutable approach, posting the state (possibly `partial`) and an instruction to a function to perform operations. Let's see!

In [4]:
testdata = """inc a
jio a, +2
tpl a
inc a""".splitlines()

inputdata = downloaded['input'].splitlines()

I felt like a convenient and customizable way to instantiate the default state:

In [5]:
class DefaultState(dict):
    def __init__(self, mapping=None, /, **kwargs):
        mymapping = {"PC": 0,
                     "a": 0,
                     "b": 0}
        if mapping is not None:
            mymapping.update(mapping)
        if kwargs:
            mymapping.update(kwargs)
        super().__init__(mymapping)

In [6]:
DefaultState(a=10)

{'PC': 0, 'a': 10, 'b': 0}

In [7]:
DefaultState(None)

{'PC': 0, 'a': 0, 'b': 0}

For a while, I thought incessant string comparisons had ruined my speed - but validating through an enum isn't a terrible idea regardless:

In [8]:
from enum import Enum, auto


class Instructions(Enum):
    HLF = "hlf"
    TPL = "tpl"
    INC = "inc"
    JMP = "jmp"
    JIE = "jie"
    JIO = "jio"

In [9]:
from IPython.display import display
foo = Instructions("hlf")
if foo == Instructions.HLF:
    display("it is half!")

'it is half!'

In my initial solution, I had each instruction as a separate function, which I could do a nifty dictionary lookup for. Then I felt like it was adding a lot of redundancy, and actually for instance the jump instructions do the same thing with different conditions (with or without a register as argument) and I wanted to get rid of the untidy incrementation of PC. That introduced a bug though (a negative conditional jump must increment PC), but eventually turned out clean enough:

In [10]:
from typing import Dict

State = Dict[str, int]


def execute_instruction(state: State, program_line: str) -> State:
    instruction_str, argument = program_line.split(' ', 1)
    instruction = Instructions(instruction_str)
    next_state = state.copy()
    if instruction in [Instructions.JMP, Instructions.JIE, Instructions.JIO]:
        if instruction == Instructions.JMP:
            offset = argument
        else:
            register, offset = argument.split(', ', 1)

        if (
                # jmp offset is a jump; it continues with the instruction
                # offset away relative to itself.
                instruction == Instructions.JMP or
                # jie r, offset is like jmp, but only jumps if register r
                # is even ("jump if even").
                (instruction == Instructions.JIE and
                 next_state[register] % 2 == 0) or
                # jio r, offset is like jmp, but only jumps if register r
                # is 1 ("jump if one", not odd).
                (instruction == Instructions.JIO and
                 next_state[register] == 1)):
            # All three jump instructions work with an offset relative to that
            # instruction. The offset is always written with a prefix + or - to
            # indicate the direction of the jump (forward or backward,
            # respectively). For example, jmp +1 would simply continue with the
            # next instruction, while jmp +0 would continuously jump back to
            # itself forever.
            next_state["PC"] += int(offset)
        else:
            next_state["PC"] += 1
    else:
        if instruction == Instructions.HLF:
            # hlf r sets register r to half its current value, then continues
            # with the next instruction.
            next_state[argument] //= 2
        elif instruction == Instructions.TPL:
            # tpl r sets register r to triple its current value, then continues
            # with the next instruction.
            next_state[argument] *= 3
        else:  # instruction == Instructions.INC, we checked above
            # inc r increments register r, adding 1 to it, then continues with
            # the next instruction.
            next_state[argument] += 1
        next_state["PC"] += 1  # Also a negative conditional jump would do this
    return next_state

In [11]:
execute_instruction(DefaultState(), "jie a, +10")

{'PC': 10, 'a': 0, 'b': 0}

This could easily be modified into a generator of some sort, it could be fun to make a visualization:

In [12]:
from typing import List


def run_program(program: List[str],
                state: State = None) -> State:
    state: State = DefaultState(state)
    while state["PC"] < len(program):
        state = execute_instruction(state, program[state["PC"]])
    return state

In [13]:
assert run_program(testdata)["a"] == 2

In [14]:
run_program(inputdata)["b"]

170

In [15]:
HTML(downloaded['part1_footer'])

## Part Two

In [16]:
HTML(downloaded['part2'])

In [17]:
run_program(inputdata, state={"a": 1})["b"]

247

In [18]:
HTML(downloaded['part2_footer'])